In [3]:
import pandas as pd
#df=pd.read_csv('https://cloud.minsa.gob.pe/s/To2QtqoNjKqobfw/download', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_VACUNACION', 'DOSIS', 'DEPARTAMENTO', 'FABRICANTE'],parse_dates=['FECHA_VACUNACION'])
df=pd.read_csv('vacunas_covid.csv', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_VACUNACION', 'DOSIS', 'DEPARTAMENTO', 'FABRICANTE'], parse_dates=['FECHA_VACUNACION'])
fecha_corte=df['FECHA_CORTE'].drop_duplicates().set_axis(['fecha_corte'])
#fecha_corte.to_json("../resultados/fecha_corte.json")

In [4]:
#DIARIO DOSIS 1 Y DOSIS 2
df_ambas_dosis=df[['FECHA_VACUNACION','DOSIS','SEXO']].groupby(['FECHA_VACUNACION','DOSIS']).count()
df_ambas_dosis=df_ambas_dosis.reset_index()
df_ambas_dosis=df_ambas_dosis.pivot(index='FECHA_VACUNACION', columns='DOSIS', values='SEXO')
df_ambas_dosis=df_ambas_dosis.rename_axis(None, axis=1)
df_ambas_dosis.columns=['DOSIS1','DOSIS2','DOSIS3']
df_ambas_dosis=df_ambas_dosis.fillna(0).astype('int')
df_ambas_dosis

,DOSIS1,DOSIS2,DOSIS3
FECHA_VACUNACION,,,
2021-02-08,4,0,0
2021-02-09,3846,0,0
2021-02-10,16582,1,0
2021-02-11,34097,0,0
2021-02-12,32365,0,0
...,...,...,...
2021-10-13,90781,101723,0
2021-10-14,75612,97424,0
2021-10-15,73687,84488,18176


In [5]:
#ACUMULADO DOSIS 1 Y DOSIS 2
df_ambas_dosis_cum=df_ambas_dosis.cumsum()
df_ambas_dosis_cum

,DOSIS1,DOSIS2,DOSIS3
FECHA_VACUNACION,,,
2021-02-08,4,0,0
2021-02-09,3850,0,0
2021-02-10,20432,1,0
2021-02-11,54529,1,0
2021-02-12,86894,1,0
...,...,...,...
2021-10-13,17437285,13666806,0
2021-10-14,17512897,13764230,0
2021-10-15,17586584,13848718,18176


In [7]:
col_poblacion=[426806,
1180638,
430736,
1497438,
668213,
1453711,
1129854,
1357075,
365317,
760267,
975182,
1361467,
2016771,
1310785,
10628470,
1027559,
173811,
192740,
271904,
2047954,
1237997,
899648,
370974,
251521,
589110]


In [10]:
# ACUMULADO POR DEPARTAMENTO (DOSIS 2 - VACUNACION COMPLETA)
df_ambas_dosis_departamento=df[['DEPARTAMENTO','DOSIS','SEXO']].groupby(['DEPARTAMENTO', 'DOSIS']).count()
df_ambas_dosis_departamento=df_ambas_dosis_departamento.reset_index()
df_ambas_dosis_departamento=df_ambas_dosis_departamento.pivot(index='DEPARTAMENTO', columns='DOSIS', values='SEXO')
df_ambas_dosis_departamento.columns=['DOSIS1','DOSIS2','DOSIS3']
df_ambas_dosis_departamento['POBLACION']=col_poblacion
df_ambas_dosis_departamento['INDICE']=round(df_ambas_dosis_departamento['DOSIS2']/(df_ambas_dosis_departamento['POBLACION']/100000)).astype('int')
df_ambas_dosis_departamento=df_ambas_dosis_departamento.fillna(0).astype('int')
df_ambas_dosis_departamento

,DOSIS1,DOSIS2,DOSIS3,POBLACION,INDICE
DEPARTAMENTO,,,,,
AMAZONAS,186520,143760,621,426806,33683
ANCASH,690183,584552,1565,1180638,49512
APURIMAC,236728,176637,0,430736,41008
AREQUIPA,899339,719627,0,1497438,48057
AYACUCHO,291990,213691,0,668213,31979
CAJAMARCA,708901,530836,908,1453711,36516
CALLAO,801576,644084,3769,1129854,57006
CUSCO,656505,516776,0,1357075,38080
HUANCAVELICA,172365,136916,648,365317,37479


In [11]:
# ACUMULADO POR GRUPO ETARIO (DOSIS 2 - VACUNACION COMPLETA)
bins = [18,30,40,50,60,70,80,df['EDAD'].max()+1]
labels = ['18 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 69 años','70 a 79 años','80 años a más']
poblacion_por_grupo_etario = [6303670,5031117,4183174,3277134,2221241,1271842,647355] 
df_edades = df
df_edades['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_edades = df_edades[df_edades.DOSIS == 2].groupby(['GRUPO_ETARIO'])["DOSIS"].count().reset_index()
df_edades.rename(columns = {'DOSIS':'DOSIS2'}, inplace = True)
df_edades['POBLACION']=poblacion_por_grupo_etario
df_edades['PORCENTAJE']=round(df_edades['DOSIS2']/df_edades['POBLACION']*100,2)
df_edades=df_edades.set_index('GRUPO_ETARIO')
df_edades

,DOSIS2,POBLACION,PORCENTAJE
GRUPO_ETARIO,,,
18 a 29 años,1911673,6303670,30.33
30 a 39 años,2872526,5031117,57.10
40 a 49 años,3090581,4183174,73.88
50 a 59 años,2588930,3277134,79.00
60 a 69 años,1864621,2221241,83.95
70 a 79 años,1075650,1271842,84.57
80 años a más,550840,647355,85.09


In [12]:
#DIARIO POR FABRICANTE
df_fabricante=df[['FECHA_VACUNACION','FABRICANTE','SEXO']].groupby(['FECHA_VACUNACION','FABRICANTE']).count()
df_fabricante=df_fabricante.reset_index()
df_fabricante=df_fabricante.pivot(index='FECHA_VACUNACION', columns='FABRICANTE', values='SEXO')
df_fabricante=df_fabricante.rename_axis(None, axis=1)
df_fabricante=df_fabricante.fillna(0).astype('int')
df_fabricante

,ASTRAZENECA,PFIZER,SINOPHARM
FECHA_VACUNACION,,,
2021-02-08,0,1,3
2021-02-09,0,0,3846
2021-02-10,0,0,16583
2021-02-11,0,1,34096
2021-02-12,0,2,32363
...,...,...,...
2021-10-13,37109,40931,114464
2021-10-14,47971,46165,78900
2021-10-15,49873,53986,72492


In [13]:
#ACUMULADO POR FABRICANTE
df_fabricante_cum=df_fabricante.cumsum()
df_fabricante_cum

,ASTRAZENECA,PFIZER,SINOPHARM
FECHA_VACUNACION,,,
2021-02-08,0,1,3
2021-02-09,0,1,3849
2021-02-10,0,1,20432
2021-02-11,0,2,54528
2021-02-12,0,4,86891
...,...,...,...
2021-10-13,1366408,16707937,13029746
2021-10-14,1414379,16754102,13108646
2021-10-15,1464252,16808088,13181138


In [14]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://www.gob.pe/pongoelhombro#contador-de-vacunados')
scrap_contador_vacunados = r.html.find('.font-bold.text-3xl')
d = []
for number in scrap_contador_vacunados:
    d.append(number.text)
contador_vacunados = pd.Series(d,index=['total_dosis1', 'total_dosis2','ayer_dosis1','ayer_dosis2','total_dosis','ayer_total_dosis','vacunacion_completa'])
#contador_vacunados.to_json('../resultados/pongo_el_hombro.json')

In [17]:
# DIARIO TACNA
df_tacna=df[df['DEPARTAMENTO'] == 'TACNA']
df_diario_tacna=df_tacna[['FECHA_VACUNACION','DOSIS','SEXO']].groupby(['FECHA_VACUNACION','DOSIS']).count()
df_diario_tacna=df_diario_tacna.reset_index()
df_diario_tacna=df_diario_tacna.pivot(index='FECHA_VACUNACION', columns='DOSIS', values='SEXO')
df_diario_tacna=df_diario_tacna.rename_axis(None, axis=1)
df_diario_tacna.columns=['DOSIS1','DOSIS2','DOSIS3']
df_diario_tacna=df_diario_tacna.fillna(0).astype('int')
df_diario_tacna

,DOSIS1,DOSIS2
FECHA_VACUNACION,,
2021-02-10,2,0
2021-02-11,724,0
2021-02-12,446,0
2021-02-13,165,0
2021-02-15,33,0
...,...,...
2021-10-13,404,2124
2021-10-14,381,1508
2021-10-15,607,1632


In [ ]:
# DIARIO PUNO
df_puno=df[df['DEPARTAMENTO'] == 'PUNO']
df_diario_puno=df_tacna[['FECHA_VACUNACION','DOSIS','SEXO']].groupby(['FECHA_VACUNACION','DOSIS']).count()
df_diario_puno=df_diario_puno.reset_index()
df_diario_puno=df_diario_puno.pivot(index='FECHA_VACUNACION', columns='DOSIS', values='SEXO')
df_diario_puno=df_diario_puno.rename_axis(None, axis=1)
df_diario_puno.columns=['DOSIS1','DOSIS2','DOSIS3']
df_diario_puno=df_diario_puno.fillna(0).astype('int')
df_diario_puno

In [ ]:
# DIARIO MADRE DE DIOS
df_mdd=df[df['DEPARTAMENTO'] == 'MADRE DE DIOS']
df_diario_mdd=df_mdd[['FECHA_VACUNACION','DOSIS','SEXO']].groupby(['FECHA_VACUNACION','DOSIS']).count()
df_diario_mdd=df_diario_mdd.reset_index()
df_diario_mdd=df_diario_mdd.pivot(index='FECHA_VACUNACION', columns='DOSIS', values='SEXO')
df_diario_mdd=df_diario_mdd.rename_axis(None, axis=1)
df_diario_mdd.columns=['DOSIS1','DOSIS2','DOSIS3']
df_diario_mdd=df_diario_mdd.fillna(0).astype('int')
df_diario_mdd

In [18]:
#TOTAL DE 11 A 16 AÑOS
bins = [11,12,13,14,15,16,17]
labels = ['11 años','12 años','13 años','14 años','15 años','16 años'] 
df_11_16 = df
df_11_16['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_11_16 = df_11_16[df_11_16.DOSIS == 2].groupby(['GRUPO_ETARIO'])["DOSIS"].count().reset_index()
df_11_16.rename(columns = {'DOSIS':'DOSIS2'}, inplace = True)
df_11_16=df_11_16.set_index('GRUPO_ETARIO')
df_11_16

,DOSIS2
GRUPO_ETARIO,
11 años,165
12 años,8602
13 años,9202
14 años,9262
15 años,9633
16 años,10407


In [19]:
#ACUMULADO DE 11 A 16 AÑOS
df_11_16_diario = df
df_11_16_diario = df[df['EDAD'].isin([11,12,13,14,15,16])]
df_11_16_diario = df_11_16_diario[df_11_16_diario.DOSIS == 2].groupby(['FECHA_VACUNACION'])["DOSIS"].count().reset_index()
df_11_16_diario = df_11_16_diario.set_index('FECHA_VACUNACION')
df_11_16_diario.rename(columns = {'DOSIS':'DOSIS2'}, inplace = True)
df_11_16_diario_cum = df_11_16_diario.cumsum()
df_11_16_diario_cum

,DOSIS2
FECHA_VACUNACION,
2021-04-29,1
2021-05-11,3
2021-05-23,4
2021-05-25,5
2021-05-26,7
...,...
2021-10-13,43917
2021-10-14,44908
2021-10-15,45677
